In [13]:
import category_encoders as ce

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Defino funciones

In [130]:
def imputar(df):
    imp = SimpleImputer()
    cols = df._get_numeric_data().columns
    df_result = df.copy()
    for x in cols:
        df_result[x]=imp.fit_transform(df[[x]])
    return df_result

In [131]:
from sklearn.metrics import mean_absolute_error

def predecir(model, train_features, train_labels, test_features, test_labels):
    predict = model.predict(test_features)
    error = mean_absolute_error(test_labels, predict)
    score = model.score(test_features,test_labels)
    
    print('Entrenamiento: {:0.4f}%'.format(model.score(train_features, train_labels)*100))
    print('Testeo: {:0.4f}%.'.format(score*100))
    print('Mean abs error: {:0.4f}.'.format(error))

    
    return predict

In [132]:
def transformar(df,colums,func):
    for x in colums:
        df[x]=df[x].transform(lambda y: func(y))
        
columnas_para_transformar = ["metroscubiertos","metrostotales","metroscubiertostotales"]

#transformar(df1,columnas_para_transformar,np.sqrt)

In [133]:
def knn_best_distance(df,dist,x_train,y_train,x_test,y_test):
    for x in dist:
        print(x+" --------------------------")
        result , k = knn_predictor(x,x_train,y_train,x_test,y_test)
        df[x] = result
        

In [134]:
from sklearn.neighbors import KNeighborsRegressor

def knn_predictor(distancia,x_train,y_train,x_test,y_test):
    kesimos = []
    result = []
    for k in range(4,10):
        kesimos.append(k)
        print(k)
        knn = KNeighborsRegressor(n_neighbors = k, metric=distancia)
        knn.fit(x_train, y_train)
        res = predecir(knn,x_train,y_train,x_test,y_test)
        result.append(res)
    return result, kesimos

## Importo dataset

In [135]:
df = pd.read_csv('../data/test_features.csv')

df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df["mes"] = df['fecha'].dt.month
df["dia"] = df['fecha'].dt.day

In [136]:
df = df.drop(columns=["direccion","descripcion","titulo"])

 ## Completo los datos faltantes

In [137]:
oneHot = ['tipodepropiedad','provincia']
binary = ['ciudad']


df_ids = df["id"]
df = df.drop(columns=["id"])

In [138]:
#Completo los numericos
df1 = imputar(df)


#Completo los categoricos
imp = SimpleImputer(strategy="constant",fill_value="-")

df1["tipodepropiedad"] = imp.fit_transform(df1[["tipodepropiedad"]])
df1["provincia"] = imp.fit_transform(df1[["provincia"]])
df1["ciudad"] = imp.fit_transform(df1[["ciudad"]])

In [139]:
df1["metroscubiertostotales"]=df1["metroscubiertos"]+df1["metrostotales"]
df1["ambientes"]=df1["banos"]+df1["habitaciones"]
df1["ambientesygarage"]=df1["banos"]+df1["habitaciones"]+df1["garages"]

 ## Genero los encodes

In [140]:
data_cols = df1._get_numeric_data().columns
baseline_data = df1[data_cols]

encoder = ce.OneHotEncoder()

encoded = encoder.fit_transform(df1[oneHot])
data = baseline_data.join(encoded)

encoder = ce.BinaryEncoder()
encoded = encoder.fit_transform(df1[binary])
data = data.join(encoded)

data.shape

(60000, 105)



## Divido data set


In [98]:
x = data[data.columns.drop("precio")]
y = data['precio']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

print(f"Original shapes: X={x.shape} y={y.shape}")
print(f"Train shapes: X={x_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={x_test.shape}  y={y_test.shape}")

Original shapes: X=(240000, 107) y=(240000,)
Train shapes: X=(192000, 107) y=(192000,)
Test  shapes: X=(48000, 107)  y=(48000,)



## Modelo y entreno

## Predicciones

In [55]:
k = list(range(4,10))
prueba = pd.DataFrame(k,index=k,columns=["Kactual"])

In [56]:
dists = ["manhattan"
]


knn_best_distance(prueba,dists,x_train,y_train,x_test,y_test)


manhattan --------------------------
Entrenamiento: 63.8971%
Testeo: 38.8238%.
Mean abs error: 1129197.8864.
Entrenamiento: 60.7249%
Testeo: 40.1496%.
Mean abs error: 1118351.0334.
Entrenamiento: 58.3783%
Testeo: 40.6936%.
Mean abs error: 1115446.7898.
Entrenamiento: 56.4860%
Testeo: 41.0841%.
Mean abs error: 1113044.2474.
Entrenamiento: 54.9238%
Testeo: 41.3226%.
Mean abs error: 1113258.1706.
Entrenamiento: 53.6208%
Testeo: 41.3020%.
Mean abs error: 1116132.9243.


# Result de test

In [126]:
#Agrego las columnas que no estan con 0's
for a in x.columns[~x.columns.isin(data.columns)]:
    data[a] = 0



#Resultado de test
knn = KNeighborsRegressor(n_neighbors = 8, metric="manhattan")

#Entreno con todo el dataset
knn.fit(x, y)

#Predigo la data
predict = knn.predict(data)

In [127]:
result = df_ids.astype("int32").to_frame()
result["target"] = predict.astype("int32")

In [128]:
result = df_ids.astype("int32").to_frame()
result["target"] = predict.astype("int32")
result.to_csv("../data/results/resultKNN_Features.csv", index=False)

In [142]:
data.shape

(60000, 107)